In [150]:
import numpy as np
import pandas as pd

In [151]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 500)

df_loan = pd.read_csv('loan.csv', low_memory=False)

In [152]:
df_loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
good_loan =  len(df_loan[(df_loan.loan_status == 'Fully Paid') |
                    (df_loan.loan_status == 'Current') | 
                    (df_loan.loan_status == 'Does not meet the credit policy. Status:Fully Paid')])
print ('Good/Bad Loan Ratio: %.2f%%'  % (good_loan/len(df_loan)*100))

Good/Bad Loan Ratio: 91.45%


In [154]:
df_loan.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [155]:
df_loan.shape

(887379, 74)

In [156]:
df_loan.drop(['id', 'member_id', 'emp_title','title','url','zip_code','verification_status','home_ownership','issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','desc','last_credit_pull_d', ], axis=1, inplace=True)

In [157]:
df_loan.count().sort_values()

dti_joint                         509
annual_inc_joint                  511
verification_status_joint         511
il_util                         18617
mths_since_rcnt_il              20810
open_acc_6m                     21372
open_il_6m                      21372
open_il_12m                     21372
open_il_24m                     21372
total_bal_il                    21372
inq_last_12m                    21372
total_cu_tl                     21372
open_rv_12m                     21372
open_rv_24m                     21372
max_bal_bc                      21372
all_util                        21372
inq_fi                          21372
mths_since_last_record         137053
mths_since_last_major_derog    221703
mths_since_last_delinq         433067
tot_cur_bal                    817103
tot_coll_amt                   817103
total_rev_hi_lim               817103
emp_length                     842554
revol_util                     886877
collections_12_mths_ex_med     887234
pub_rec     

In [158]:
lack_of_data_idx = [x for x in df_loan.count() < 887379*0.2]
df_loan.drop(df_loan.columns[lack_of_data_idx], 1, inplace=True)

In [160]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 42 columns):
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_length                     842554 non-null object
annual_inc                     887375 non-null float64
loan_status                    887379 non-null object
pymnt_plan                     887379 non-null object
purpose                        887379 non-null object
addr_state                     887379 non-null object
dti                            887379 non-null float64
delinq_2yrs                    887350 non-null float64
inq_last_6mths   

In [162]:
print (df_loan.mths_since_last_delinq.min(), df_loan.mths_since_last_delinq.max())
print(df_loan.mths_since_last_delinq.mean())
print(df_loan.mths_since_last_delinq.median())

0.0 188.0
34.06379844227337
31.0


In [163]:
df_loan.mths_since_last_delinq = df_loan.mths_since_last_delinq.fillna(df_loan.mths_since_last_delinq.median())

In [164]:
df_loan.dropna(inplace=True)

In [165]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208616 entries, 42538 to 887377
Data columns (total 42 columns):
loan_amnt                      208616 non-null float64
funded_amnt                    208616 non-null float64
funded_amnt_inv                208616 non-null float64
term                           208616 non-null object
int_rate                       208616 non-null float64
installment                    208616 non-null float64
grade                          208616 non-null object
sub_grade                      208616 non-null object
emp_length                     208616 non-null object
annual_inc                     208616 non-null float64
loan_status                    208616 non-null object
pymnt_plan                     208616 non-null object
purpose                        208616 non-null object
addr_state                     208616 non-null object
dti                            208616 non-null float64
delinq_2yrs                    208616 non-null float64
inq_last_6mth

In [166]:
# Calculate Good and Bad Loan Status Ratio
good_loan =  len(df_loan[(df_loan.loan_status == 'Fully Paid') |
                    (df_loan.loan_status == 'Current') | 
                    (df_loan.loan_status == 'Does not meet the credit policy. Status:Fully Paid')])
print ('Good/Bad Loan Ratio: %.2f%%'  % (good_loan/len(df_loan)*100))

Good/Bad Loan Ratio: 92.09%


In [167]:
df_loan['good_loan'] = np.where((df_loan.loan_status == 'Fully Paid') |
                        (df_loan.loan_status == 'Current') | 
                        (df_loan.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 1, 0)

In [168]:
df_loan.shape

(208616, 43)

In [169]:
columns = ['term', 'grade', 'sub_grade', 'emp_length', 'purpose', 'application_type','addr_state',
           'pymnt_plan', 'initial_list_status']

for col in columns:
    tmp_df = pd.get_dummies(df_loan[col], prefix=col)
    df_loan = pd.concat((df_loan, tmp_df), axis=1)

In [170]:
df_loan.drop(['loan_status',
           'term',
           'grade',
           'sub_grade',
           'emp_length',
           'addr_state',
           'initial_list_status',
           'pymnt_plan',
           'purpose',
           'application_type'], axis=1, inplace=True)

In [171]:
df_loan = df_loan.rename(columns= {'emp_length_< 1 year':'emp_length_lt_1 year',
                         'emp_length_n/a':'emp_length_na'})

In [172]:
df_loan.shape

(208616, 158)

In [173]:
df_loan.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,good_loan,term_ 36 months,term_ 60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_lt_1 year,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_INDIVIDUAL,application_type_JOINT,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,pymnt_plan_n,pymnt_plan_y,initial_list_status_f,initial_list_status_w
42538,12000.0,12000.0,12000.0,13.53,407.40,40000.0,16.94,0.0,0.0,53.0,7.0,2.0,5572.0,68.8,32.0,0.00,0.00,13359.770000,13359.77,11999.99,1359.78,0.0,0.0,0.0,119.17,0.0,53.0,1.0,0.0,15386.0,13605.0,8100.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
42539,15000.0,15000.0,15000.0,8.90,476.30,63000.0,16.51,0.0,0.0,34.0,8.0,0.0,11431.0,74.2,29.0,5449.27,5449.27,11431.200000,11431.20,9550.73,1880.47,0.0,0.0,0.0,476.30,0.0,34.0,1.0,0.0,1514.0,272492.0,15400.0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
42543,10000.0,10000.0,10000.0,9.67,321.13,102000.0,15.55,2.0,0.0,11.0,9.0,0.0,9912.0,44.4,22.0,3659.12,3659.12,7706.810000,7706.81,6340.88,1365.93,0.0,0.0,0.0,321.13,0.0,54.0,1.0,0.0,0.0,39143.0,22300.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
42544,3000.0,3000.0,3000.0,12.85,100.87,25000.0,24.68,0.0,0.0,58.0,5.0,2.0,2875.0,54.2,26.0,0.00,0.00,3181.532981,3181.53,3000.00,181.53,0.0,0.0,0.0,2677.23,0.0,69.0,1.0,0.0,154.0,19530.0,5300.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [174]:
from sklearn.model_selection import train_test_split

y = df_loan['good_loan']
X = df_loan.ix[:, df_loan.columns != 'good_loan']

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2, random_state=44)

D:\Programas_D\Anaconda3\envs\normal\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [175]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(166892, 157)
(41724, 157)
(166892,)
(41724,)


In [176]:
X.shape

(208616, 157)

In [177]:
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
rob_scaler = RobustScaler()

X_train_R = rob_scaler.fit_transform(X_train)
X_test_R = rob_scaler.transform(X_test)

In [178]:
from sklearn.svm import SVC

In [179]:
# Weighted prediction feature
y_0 = len(y_train[y_train == 0])/len(y_train)
y_1 = 1 - y_0

### Random Forest

In [180]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 20)
rf.fit(X_train, y_train)
       
rf_predictions = rf.predict(X_test)

#print(rf.score(X_test, y_test))
scores = cross_val_score(rf, X_test, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, rf_predictions))
print(mt.confusion_matrix(y_test, rf_predictions))

[0.95937687 0.96153385 0.95626123 0.9602157  0.95841323]
Accuracy: 0.96 (+/- 0.00)

             precision    recall  f1-score   support

          0       0.85      0.59      0.70      3240
          1       0.97      0.99      0.98     38484

avg / total       0.96      0.96      0.96     41724

[[ 1913  1327]
 [  331 38153]]


### XGBOOST

In [181]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

xgb_predictions = xgb.predict(X_test)
                            
#print(xgb.score(X_test, y_test))
scores = cross_val_score(xgb, X_test, y_test, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

print(mt.classification_report(y_test, xgb_predictions))
print(mt.confusion_matrix(y_test, xgb_predictions))

D:\Programas_D\Anaconda3\envs\normal\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Programas_D\Anaconda3\envs\normal\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Programas_D\Anaconda3\envs\normal\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Programas_D\Anaconda3\envs\normal\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an 

[0.96033553 0.96249251 0.95793889 0.96213301 0.95841323]
Accuracy: 0.96 (+/- 0.00)

             precision    recall  f1-score   support

          0       0.89      0.57      0.69      3240
          1       0.96      0.99      0.98     38484

avg / total       0.96      0.96      0.96     41724

[[ 1841  1399]
 [  225 38259]]


D:\Programas_D\Anaconda3\envs\normal\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
